# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/PINN

/content/drive/.shortcut-targets-by-id/1kBpsdlzoY9Dl5Og9N1a-GxSNoIkIrQV4/PINN


In [3]:
!ls -l

total 3116
drwx------ 2 root root   4096 Feb 28 00:25  ar_377_2011-02-12T00:00:00
drwx------ 2 root root   4096 Feb  6 01:28  AR_377_series
drwx------ 2 root root   4096 Feb 27 07:32  ar_7115_2017-09-06T08:36:00
drwx------ 2 root root   4096 Feb  6 07:56  AR_7115_series
drwx------ 2 root root   4096 Feb 28 00:26  ar_series_377_2011-02-12T00:00:00
drwx------ 2 root root   4096 Feb  7 23:25  ar_series_7115_2017-09-06T08:36:00
drwx------ 2 root root   4096 Feb 22 23:19  author
-rw------- 1 root root 624555 Feb 28 06:10 'extrapolation AR377.ipynb'
-rw------- 1 root root 723048 Feb 27 07:32 'extrapolation AR7115.ipynb'
-rw------- 1 root root 771376 Feb 28 00:26  extrapolation_custom.ipynb
-rw------- 1 root root  56254 Feb 27 07:31  hmi_cropped_binning.png
-rw------- 1 root root  56881 Feb 27 07:31  hmi_cropped.png
-rw------- 1 root root  56881 Feb 27 07:31  hmi_original.png
-rw------- 1 root root  17351 Feb 27 01:53  nf2_to_info.ipynb
-rw------- 1 root root 853125 Feb 26 23:14 'npy vs nc.ip

In [4]:
!pip install -q git+https://github.com/RobertJaro/NF2.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.6 MB/s eta 0:00:00


In [5]:
!pip install -q vtk mayavi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 38.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.2/229.2 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.9/281.9 KB 32.8 MB/s eta 0:00:00


In [6]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from tvtk.api import tvtk, write_data
from nf2.potential.potential_field import get_potential
from datetime import datetime

In [7]:
def make_info(nf2_file, vtk_path, npy_path, npy_pot_path):
  device = torch.device("cuda") if torch.backends.mps.is_available() else "cpu"
  state = torch.load(str(nf2_file), map_location=device)
  meta_info = state['meta_info']

  # Number
  print(meta_info['harpnum'])
  print(meta_info['noaa_ar'])
  # Observation
  print(meta_info['date-obs'])
  print(meta_info['t_obs'])
  print(meta_info['t_rec'])
  # shape
  print(state['cube_shape'])
  print(meta_info['naxis1'], meta_info['naxis2'])

  # nf2 -> b
  model = torch.nn.DataParallel(state['model'])
  cube_shape = state['cube_shape']
  z = cube_shape[2]
  coords = np.stack(np.mgrid[:cube_shape[0]:1, :cube_shape[1]:1, :z:1], -1)
  spatial_norm = 160
  coords = torch.tensor(coords / spatial_norm, dtype=torch.float32)
  coords_shape = coords.shape
  cube = []
  batch_size = 1000
  coords = coords.view((-1, 3))
  it = range(int(np.ceil(coords.shape[0] / batch_size)))
  for k in tqdm(it):
      coord = coords[k * batch_size: (k + 1) * batch_size]
      coord = coord.to(device)
      coord.requires_grad = True
      cube += [model(coord).detach().cpu()]
  cube = torch.cat(cube)
  cube = cube.view(*coords_shape).numpy()
  b_norm = 2500
  b = cube * b_norm    

  # b -> vtk
  bin = 2
  Mm_per_pix = 360e-3 * bin
  # Unpack
  dim = b.shape[:-1]
  # Generate the grid
  pts = np.stack(np.mgrid[0:dim[0], 0:dim[1], 0:dim[2]], -1).astype(np.int64) * Mm_per_pix
  # reorder the points and vectors in agreement with VTK
  # requirement of x first, y next and z last.
  pts = pts.transpose(2, 1, 0, 3)
  pts = pts.reshape((-1, 3))
  vectors = b.transpose(2, 1, 0, 3)
  vectors = vectors.reshape((-1, 3))
  sg = tvtk.StructuredGrid(dimensions=dim, points=pts)
  sg.point_data.vectors = vectors
  sg.point_data.vectors.name = 'B'
  write_data(sg, str(vtk_path))
  print(str(vtk_path))

  # b -> npy
  np.save(str(npy_path), b)
  print(str(npy_path))

  # b -> potential b
  potential = get_potential(b[:, :, 0, 2], b.shape[2], batch_size=int(1e3))
  b_potential = - 1 * np.stack(np.gradient(potential, axis=[0, 1, 2], edge_order=2), axis=-1)
  np.save(str(npy_pot_path), b_potential)
  print(str(npy_pot_path))

In [8]:
def energy_series_csv(series_base_path):
  energy_path = series_base_path / 'eval_energy'
  energy_files = [x for x in energy_path.glob('**/*.npy')]
  energy_files = sorted(energy_files)
  energy_series_dates = [datetime.strptime(os.path.basename(f), '%Y%m%d_%H%M%S_TAI.npy') for f in energy_files]
  energy_density = np.array([np.load(f) for f in energy_files])
  df = pd.DataFrame({"date":energy_series_dates, "energy_density":energy_density})
  csv_path = series_base_path / 'energy.csv'
  df.to_csv(str(csv_path), index=False)  
  print(str(csv_path))

In [9]:
def free_energy_series_csv(series_base_path):
  free_energy_path = series_base_path / 'eval_free_energy'
  free_energy_files = [x for x in free_energy_path.glob('**/*.npy')]
  free_energy_files = sorted(free_energy_files)
  free_energy_series_dates = [datetime.strptime(os.path.basename(f), '%Y%m%d_%H%M%S_TAI.npy') for f in free_energy_files]
  free_energy_density = np.array([np.load(f) for f in free_energy_files])
  df = pd.DataFrame({"date":free_energy_series_dates, "free_energy_density":free_energy_density})
  csv_path = series_base_path / 'free_energy.csv'
  df.to_csv(str(csv_path), index=False)  
  print(str(csv_path))

# NOAA 11158

In [10]:
series_base_path = Path('ar_series_377_2011-02-12T00:00:00')
nf2_file = series_base_path / 'base/20110215_000000_TAI/extrapolation_result.nf2'
vtk_path = series_base_path / '11158_20110215_000000_colab.vtk'
npy_path = series_base_path / '11158_20110215_000000_colab.npy'
npy_pot_path = series_base_path / '11158_20110215_000000_colab_pot.npy'

In [11]:
make_info(nf2_file, vtk_path, npy_path, npy_pot_path)

377
11158
2011-02-14T23:58:12.300
2011.02.14_23:59:53.752_TAI
2011.02.15_00:00:00.000_TAI
[296, 184, 160]
744 377


100%|██████████| 8715/8715 [00:18<00:00, 473.67it/s]


ar_series_377_2011-02-12T00:00:00/11158_20110215_000000_colab.vtk
ar_series_377_2011-02-12T00:00:00/11158_20110215_000000_colab.npy


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Potential Field: 100%|██████████| 8715/8715 [03:39<00:00, 39.63it/s]


ar_series_377_2011-02-12T00:00:00/11158_20110215_000000_colab_pot.npy


In [12]:
energy_series_csv(series_base_path)
free_energy_series_csv(series_base_path)

ar_series_377_2011-02-12T00:00:00/energy.csv
ar_series_377_2011-02-12T00:00:00/free_energy.csv


# NOAA 12673

## author

In [ ]:
series_base_path = Path('ar_series_7115_2017-09-06T08:36:00')
nf2_file = series_base_path / 'example_7115.nf2'
vtk_path = series_base_path / '12673_20170906_083600_author.vtk'
npy_path = series_base_path / '12673_20170906_083600_author.npy'
npy_pot_path = series_base_path / '12673_20170906_083600_author_pot.npy'

In [ ]:
make_info(nf2_file, vtk_path, npy_path, npy_pot_path)

7115
12673
2017-09-06T08:34:42.100
2017.09.06_08:36:04.065_TAI
2017.09.06_08:36:00.000_TAI
[344, 224, 160]
688 448


100%|██████████| 12329/12329 [00:28<00:00, 435.06it/s]


ar_series_7115_2017-09-06T08:36:00/12673_20170906_083600_author.vtk
ar_series_7115_2017-09-06T08:36:00/12673_20170906_083600_author.npy


Potential Field: 100%|██████████| 12329/12329 [07:20<00:00, 28.01it/s]


ar_series_7115_2017-09-06T08:36:00/12673_20170906_083600_author_pot.npy


In [ ]:
def author_series(series_base_path):
  start_with_author_series = series_base_path / 'start_with_author_series'
  energy_path = start_with_author_series / 'eval_energy'
  energy_files = [x for x in energy_path.glob('**/*.npy')]
  energy_files = sorted(energy_files)
  energy_series_dates = [datetime.strptime(os.path.basename(f), '%Y%m%d_%H%M%S_TAI.npy') for f in energy_files]
  energy_density = np.array([np.load(f).sum() for f in energy_files])
  df = pd.DataFrame({"date":energy_series_dates, "energy_density":energy_density})
  csv_path = series_base_path / 'energy_author.csv'
  df.to_csv(csv_path, index=False)  

  free_energy_path = start_with_author_series / 'eval_free_energy'
  free_energy_files = [x for x in free_energy_path.glob('**/*.npy')]
  free_energy_files = sorted(free_energy_files)
  free_energy_series_dates = [datetime.strptime(os.path.basename(f), '%Y%m%d_%H%M%S_TAI.npy') for f in free_energy_files]
  free_energy_density = np.array([np.load(f).sum() for f in free_energy_files])
  df = pd.DataFrame({"date":free_energy_series_dates, "free_energy_density":free_energy_density})
  csv_path = series_base_path / 'free_energy_author.csv'
  df.to_csv(csv_path, index=False)  

In [ ]:
author_series(series_base_path)

## colab

In [ ]:
series_base_path = Path('ar_series_7115_2017-09-06T08:36:00')
nf2_file = series_base_path / 'base/20170906_083600_TAI/extrapolation_result.nf2'
vtk_path = series_base_path / '12673_20170906_083600_colab.vtk'
npy_path = series_base_path / '12673_20170906_083600_colab.npy'
npy_pot_path = series_base_path / '12673_20170906_083600_colab_pot.npy'

In [ ]:
make_info(nf2_file, vtk_path, npy_path, npy_pot_path)

7115
12673
2017-09-06T08:34:42.100
2017.09.06_08:36:04.065_TAI
2017.09.06_08:36:00.000_TAI
[344, 224, 160]
688 448


100%|██████████| 12329/12329 [00:19<00:00, 645.66it/s]


ar_series_7115_2017-09-06T08:36:00/12673_20170906_083600_colab.vtk
ar_series_7115_2017-09-06T08:36:00/12673_20170906_083600_colab.npy


Potential Field: 100%|██████████| 12329/12329 [07:18<00:00, 28.12it/s]


ar_series_7115_2017-09-06T08:36:00/12673_20170906_083600_colab_pot.npy


In [ ]:
energy_series_csv(series_base_path)
free_energy_series_csv(series_base_path)

ar_series_7115_2017-09-06T08:36:00/energy.csv
ar_series_7115_2017-09-06T08:36:00/free_energy.csv
